In [12]:
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
client_id = r'fd388b96c2804acc849e66d03c898c07'
client_secret = r'6971e5936216cae8695169ea88f866b0e6e277069f58419bff49d9c131ca8510'

auth_data = {
    'grant_type'    : 'client_credentials',
    'client_id'     : client_id,
    'client_secret' : client_secret,
    'scope'         : 'read_content read_financial_data read_product_data read_user_profile'
}

# create session instance
session = requests.Session()

# update session headers
session.headers.update({'Authorization':'Bearer '+ '56xlmtJK8dcsTty7ejMKkpRwFI8y'})
asset_id = 'MA4B66MW5E27U8P32SB'

request_url = "https://api.marquee.gs.com/v1/data/TREOD/query"

request_query = {
                    "where": {
                        "assetId": ["MA4B66MW5E27UANEQ6Q"]
                    },
                    "startDate": "2017-01-15",
                    "endDate":"2018-01-15"
               }

request = session.post(url=request_url, json=request_query)
results = json.loads(request.text)
table = pd.DataFrame(results['data'])

In [3]:
table

,adjustedAskPrice,adjustedBidPrice,adjustedHighPrice,adjustedLowPrice,adjustedOpenPrice,adjustedTradePrice,adjustedVolume,askPrice,assetId,bidPrice,date,highPrice,lowPrice,openPrice,tradePrice,updateTime,volume
0,235.84,235.75,239.9600,234.370,236.70,235.58,4617522.0,235.84,MA4B66MW5E27UANEQ6Q,235.75,2017-01-17,239.9600,234.370,236.70,235.58,2017-01-18T02:40:00Z,4617522.0
1,238.35,238.31,239.7100,235.580,236.65,238.36,3768967.0,238.35,MA4B66MW5E27UANEQ6Q,238.31,2017-01-18,239.7100,235.580,236.65,238.36,2017-01-19T02:33:00Z,3768967.0
2,243.76,243.62,248.6800,240.750,247.25,243.76,7732303.0,243.76,MA4B66MW5E27UANEQ6Q,243.62,2017-01-19,248.6800,240.750,247.25,243.76,2017-01-20T02:28:00Z,7732303.0
3,244.71,244.59,246.0000,243.010,245.46,244.73,4204275.0,244.71,MA4B66MW5E27UANEQ6Q,244.59,2017-01-20,246.0000,243.010,245.46,244.73,2017-01-21T03:14:00Z,4204275.0
4,248.92,248.86,250.8899,245.500,245.85,248.92,6262938.0,248.92,MA4B66MW5E27UANEQ6Q,248.86,2017-01-23,250.8899,245.500,245.85,248.92,2017-01-24T02:43:00Z,6262938.0
5,254.59,254.52,254.8000,249.650,250.00,254.61,4965451.0,254.59,MA4B66MW5E27UANEQ6Q,254.52,2017-01-24,254.8000,249.650,250.00,254.61,2017-01-25T02:44:00Z,4965451.0
6,254.40,254.31,258.4600,251.800,257.31,254.47,5146361.0,254.40,MA4B66MW5E27UANEQ6Q,254.31,2017-01-25,258.4600,251.800,257.31,254.47,2017-01-26T02:39:00Z,5146361.0
7,252.49,252.42,255.7400,250.750,254.29,252.51,3152123.0,252.49,MA4B66MW5E27UANEQ6Q,252.42,2017-01-26,255.7400,250.750,254.29,252.51,2017-01-27T02:43:00Z,3152123.0
8,252.96,252.87,253.0000,248.520,251.38,252.95,3166336.0,252.96,MA4B66MW5E27UANEQ6Q,252.87,2017-01-27,253.0000,248.520,251.38,252.95,2017-01-28T03:10:00Z,3166336.0
9,250.64,250.62,255.2899,247.100,252.53,250.63,3801074.0,250.64,MA4B66MW5E27UANEQ6Q,250.62,2017-01-30,255.2899,247.100,252.53,250.63,2017-01-31T02:45:00Z,3801074.0


In [29]:
tesla = table[['date', 'adjustedHighPrice', 'adjustedLowPrice', 'adjustedOpenPrice', 'tradePrice', 'volume', 'adjustedTradePrice']]

In [30]:
tesla.columns = ['Date', 'High','Low', 'Open', 'Close', 'Volume', 'Adj Close']
tesla.set_index('Date', inplace=True)
tesla.index = pd.to_datetime(tesla.index)

In [31]:
print(tesla)

                High      Low    Open   Close      Volume  Adj Close
Date                                                                
2017-01-17  239.9600  234.370  236.70  235.58   4617522.0     235.58
2017-01-18  239.7100  235.580  236.65  238.36   3768967.0     238.36
2017-01-19  248.6800  240.750  247.25  243.76   7732303.0     243.76
2017-01-20  246.0000  243.010  245.46  244.73   4204275.0     244.73
2017-01-23  250.8899  245.500  245.85  248.92   6262938.0     248.92
2017-01-24  254.8000  249.650  250.00  254.61   4965451.0     254.61
2017-01-25  258.4600  251.800  257.31  254.47   5146361.0     254.47
2017-01-26  255.7400  250.750  254.29  252.51   3152123.0     252.51
2017-01-27  253.0000  248.520  251.38  252.95   3166336.0     252.95
2017-01-30  255.2899  247.100  252.53  250.63   3801074.0     250.63
2017-01-31  255.8900  247.700  249.24  251.93   4116104.0     251.93
2017-02-01  253.2000  249.050  253.05  249.24   3958829.0     249.24
2017-02-02  252.4200  247.710  248

In [35]:
#### Trading Strategy ####
def trading_strategy():
    # Initialize the short and long windows
    short_window = 40
    long_window = 100
    # Initialize the `signals` DataFrame with the `signal` column
    signals = pd.DataFrame(index=tesla.index)
    signals['signal'] = 0.0
    # Create short simple moving average over the short window
    signals['short_mavg'] = tesla['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
    # Create long simple moving average over the long window
    signals['long_mavg'] = tesla['Close'].rolling(window=long_window, min_periods=1, center=False).mean()
    # Create signals
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] 
                                                > signals['long_mavg'][short_window:], 1.0, 0.0)   
    # Generate trading orders
    signals['positions'] = signals['signal'].diff()
    
    return signals


#### Backtesting ####
def backtesting(signals):
    # Set the initial capital
    initial_capital= float(100000.0)
    # Create a DataFrame `positions`
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    # Buy a 100 shares
    positions['TSLA'] = 100*signals['signal']   
    # Initialize the portfolio with value owned   
    portfolio = positions.multiply(tesla['Adj Close'], axis=0)
    # Store the difference in shares owned 
    pos_diff = positions.diff()
    # Add `holdings` to portfolio
    portfolio['holdings'] = (positions.multiply(tesla['Adj Close'], axis=0)).sum(axis=1)
    # Add `cash` to portfolio
    portfolio['cash'] = initial_capital - (pos_diff.multiply(tesla['Adj Close'], axis=0)).sum(axis=1).cumsum()   
    # Add `total` to portfolio
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']
    # Add `returns` to portfolio
    portfolio['returns'] = portfolio['total'].pct_change()
    
    return portfolio


#### Sharpe Ratio ####
def sharpe_ratio(portfolio):
    # Isolate the returns of your strategy
    returns = portfolio['returns']
    # annualized Sharpe ratio
    sharpe_ratio = np.sqrt(252) * (returns.mean() / returns.std())
    
    return sharpe_ratio


#### Maximum Drawdown ####
def maximum_drawdown():
    # Define a trailing 252 trading day window
    window = 252
    # Calculate the max drawdown in the past window days for each day
    rolling_max = tesla['Adj Close'].rolling(window, min_periods=1).max()
    daily_drawdown = tesla['Adj Close']/rolling_max - 1.0
    # Calculate the minimum (negative) daily drawdown
    max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()
    
    return max_daily_drawdown


#### Compound Annual Growth Rate ####
def cagr():
    # Get the number of days in `aapl`
    days = (tesla.index[-1] - tesla.index[0]).days
    # Calculate the CAGR 
    cagr = ((((tesla['Adj Close'][-1]) / tesla['Adj Close'][1])) ** (365.0/days)) - 1
    
    return cagr

def extract_signals(signals, indicator):
    locs = signals.loc[signals.positions == indicator].index
    prices = signals.short_mavg[signals.positions == indicator]
    
    return locs, signals
    
    
signals = trading_strategy()
portfolio = backtesting(signals)
sharpe_ratio = sharpe_ratio(portfolio)
maximum_drawdown = maximum_drawdown()
cagr = cagr()